<a href="https://colab.research.google.com/github/AsfandYar-Ktk/0-9-Digit-Classification-Web-App/blob/main/Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('/content/dataset_traffic_accident_prediction1.csv')
print(df)

    Weather   Road_Type Time_of_Day  Traffic_Density  Speed_Limit  \
0     Rainy   City Road     Morning              1.0        100.0   
1     Clear  Rural Road       Night              NaN        120.0   
2     Rainy     Highway     Evening              1.0         60.0   
3     Clear   City Road   Afternoon              2.0         60.0   
4     Rainy     Highway     Morning              1.0        195.0   
..      ...         ...         ...              ...          ...   
835   Clear     Highway       Night              2.0         30.0   
836   Rainy  Rural Road     Evening              2.0         60.0   
837   Foggy     Highway     Evening              NaN         30.0   
838   Foggy     Highway   Afternoon              2.0         60.0   
839   Clear     Highway   Afternoon              1.0         60.0   

     Number_of_Vehicles  Driver_Alcohol Accident_Severity      Road_Condition  \
0                   5.0             0.0               NaN                 Wet   
1        

# Preprocessing

For now I have removed the NaN valued rows. In future pre-process the missing values better.

In [68]:
df_copy = df.copy()

In [69]:
df_copy.Accident.value_counts()

,count
Accident,
0.0,559
1.0,239


In [70]:
df_copy['Accident'] = [0 if (x == None) else x for x in df_copy['Accident']]

Convert string values to number

In [71]:
df_copy['Weather'].value_counts()

,count
Weather,
Clear,334
Rainy,230
Foggy,107
Snowy,87
Stormy,40


In [72]:
weather_dummies = pd.get_dummies(df_copy['Weather'], prefix='Weather')
print(weather_dummies)

     Weather_Clear  Weather_Foggy  Weather_Rainy  Weather_Snowy  \
0            False          False           True          False   
1             True          False          False          False   
2            False          False           True          False   
3             True          False          False          False   
4            False          False           True          False   
..             ...            ...            ...            ...   
835           True          False          False          False   
836          False          False           True          False   
837          False           True          False          False   
838          False           True          False          False   
839           True          False          False          False   

     Weather_Stormy  
0             False  
1             False  
2             False  
3             False  
4             False  
..              ...  
835           False  
836           False

In [73]:
df_copy = pd.concat([df_copy, weather_dummies], axis = 1)
print(df_copy.head())

  Weather   Road_Type Time_of_Day  Traffic_Density  Speed_Limit  \
0   Rainy   City Road     Morning              1.0        100.0   
1   Clear  Rural Road       Night              NaN        120.0   
2   Rainy     Highway     Evening              1.0         60.0   
3   Clear   City Road   Afternoon              2.0         60.0   
4   Rainy     Highway     Morning              1.0        195.0   

   Number_of_Vehicles  Driver_Alcohol Accident_Severity      Road_Condition  \
0                 5.0             0.0               NaN                 Wet   
1                 3.0             0.0          Moderate                 Wet   
2                 4.0             0.0               Low                 Icy   
3                 3.0             0.0               Low  Under Construction   
4                11.0             0.0               Low                 Dry   

  Vehicle_Type  Driver_Age  Driver_Experience Road_Light_Condition  Accident  \
0          Car        51.0               4

In [74]:
df_copy = df_copy.drop('Weather', axis = 1)
print(df_copy.head())

    Road_Type Time_of_Day  Traffic_Density  Speed_Limit  Number_of_Vehicles  \
0   City Road     Morning              1.0        100.0                 5.0   
1  Rural Road       Night              NaN        120.0                 3.0   
2     Highway     Evening              1.0         60.0                 4.0   
3   City Road   Afternoon              2.0         60.0                 3.0   
4     Highway     Morning              1.0        195.0                11.0   

   Driver_Alcohol Accident_Severity      Road_Condition Vehicle_Type  \
0             0.0               NaN                 Wet          Car   
1             0.0          Moderate                 Wet        Truck   
2             0.0               Low                 Icy          Car   
3             0.0               Low  Under Construction          Bus   
4             0.0               Low                 Dry          Car   

   Driver_Age  Driver_Experience Road_Light_Condition  Accident  \
0        51.0            

One hot encode Road_Type

In [75]:
df_copy['Road_Type'].value_counts()

,count
Road_Type,
Highway,402
City Road,230
Rural Road,125
Mountain Road,41


In [76]:
road_dummies = pd.get_dummies(df_copy['Road_Type'], prefix="Road")
print(road_dummies)

     Road_City Road  Road_Highway  Road_Mountain Road  Road_Rural Road
0              True         False               False            False
1             False         False               False             True
2             False          True               False            False
3              True         False               False            False
4             False          True               False            False
..              ...           ...                 ...              ...
835           False          True               False            False
836           False         False               False             True
837           False          True               False            False
838           False          True               False            False
839           False          True               False            False

[840 rows x 4 columns]


In [77]:
road_dummies = road_dummies.rename(columns={'Road_City Road': 'Road_City_Road'})

In [78]:
road_dummies = road_dummies.rename(columns={'Road_Mountain Road': 'Road_Mountain_Road'})

In [79]:
road_dummies = road_dummies.rename(columns={'Road_Rural Road': 'Road_Rural_Road'})

In [80]:
road_dummies

,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road
0,True,False,False,False
1,False,False,False,True
2,False,True,False,False
3,True,False,False,False
4,False,True,False,False
...,...,...,...,...
835,False,True,False,False
836,False,False,False,True
837,False,True,False,False
838,False,True,False,False


In [81]:
df_copy = pd.concat([df_copy, road_dummies], axis=1)
df_copy

,Road_Type,Time_of_Day,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,...,Accident,Weather_Clear,Weather_Foggy,Weather_Rainy,Weather_Snowy,Weather_Stormy,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road
0,City Road,Morning,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,...,0.0,False,False,True,False,False,True,False,False,False
1,Rural Road,Night,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,...,0.0,True,False,False,False,False,False,False,False,True
2,Highway,Evening,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,...,0.0,False,False,True,False,False,False,True,False,False
3,City Road,Afternoon,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,...,0.0,True,False,False,False,False,True,False,False,False
4,Highway,Morning,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,...,1.0,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Highway,Night,2.0,30.0,4.0,0.0,Low,Dry,Car,23.0,...,0.0,True,False,False,False,False,False,True,False,False
836,Rural Road,Evening,2.0,60.0,4.0,0.0,Low,Dry,Motorcycle,52.0,...,1.0,False,False,True,False,False,False,False,False,True
837,Highway,Evening,NaN,30.0,4.0,1.0,High,Dry,Car,NaN,...,NaN,False,True,False,False,False,False,True,False,False
838,Highway,Afternoon,2.0,60.0,3.0,0.0,Low,Dry,Car,25.0,...,0.0,False,True,False,False,False,False,True,False,False


In [82]:
df_copy = df_copy.drop('Road_Type', axis=1)
df_copy

,Time_of_Day,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,...,Accident,Weather_Clear,Weather_Foggy,Weather_Rainy,Weather_Snowy,Weather_Stormy,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road
0,Morning,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,...,0.0,False,False,True,False,False,True,False,False,False
1,Night,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,...,0.0,True,False,False,False,False,False,False,False,True
2,Evening,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,...,0.0,False,False,True,False,False,False,True,False,False
3,Afternoon,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,...,0.0,True,False,False,False,False,True,False,False,False
4,Morning,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,...,1.0,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Night,2.0,30.0,4.0,0.0,Low,Dry,Car,23.0,15.0,...,0.0,True,False,False,False,False,False,True,False,False
836,Evening,2.0,60.0,4.0,0.0,Low,Dry,Motorcycle,52.0,46.0,...,1.0,False,False,True,False,False,False,False,False,True
837,Evening,NaN,30.0,4.0,1.0,High,Dry,Car,NaN,34.0,...,NaN,False,True,False,False,False,False,True,False,False
838,Afternoon,2.0,60.0,3.0,0.0,Low,Dry,Car,25.0,19.0,...,0.0,False,True,False,False,False,False,True,False,False


One hot encode Time_of_day

In [83]:
df_copy['Time_of_Day'].value_counts()

,count
Time_of_Day,
Afternoon,272
Evening,218
Morning,201
Night,107


In [84]:
time_dummies = pd.get_dummies(df_copy['Time_of_Day'], prefix='Time')
print(time_dummies)

     Time_Afternoon  Time_Evening  Time_Morning  Time_Night
0             False         False          True       False
1             False         False         False        True
2             False          True         False       False
3              True         False         False       False
4             False         False          True       False
..              ...           ...           ...         ...
835           False         False         False        True
836           False          True         False       False
837           False          True         False       False
838            True         False         False       False
839            True         False         False       False

[840 rows x 4 columns]


In [85]:
df_copy = pd.concat([df_copy, time_dummies], axis=1)

In [86]:
df_copy

,Time_of_Day,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,...,Weather_Snowy,Weather_Stormy,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,Morning,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,...,False,False,True,False,False,False,False,False,True,False
1,Night,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,...,False,False,False,False,False,True,False,False,False,True
2,Evening,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,...,False,False,False,True,False,False,False,True,False,False
3,Afternoon,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,...,False,False,True,False,False,False,True,False,False,False
4,Morning,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Night,2.0,30.0,4.0,0.0,Low,Dry,Car,23.0,15.0,...,False,False,False,True,False,False,False,False,False,True
836,Evening,2.0,60.0,4.0,0.0,Low,Dry,Motorcycle,52.0,46.0,...,False,False,False,False,False,True,False,True,False,False
837,Evening,NaN,30.0,4.0,1.0,High,Dry,Car,NaN,34.0,...,False,False,False,True,False,False,False,True,False,False
838,Afternoon,2.0,60.0,3.0,0.0,Low,Dry,Car,25.0,19.0,...,False,False,False,True,False,False,True,False,False,False


In [87]:
df_copy = df_copy.drop('Time_of_Day', axis=1)
print(df_copy.head())

   Traffic_Density  Speed_Limit  Number_of_Vehicles  Driver_Alcohol  \
0              1.0        100.0                 5.0             0.0   
1              NaN        120.0                 3.0             0.0   
2              1.0         60.0                 4.0             0.0   
3              2.0         60.0                 3.0             0.0   
4              1.0        195.0                11.0             0.0   

  Accident_Severity      Road_Condition Vehicle_Type  Driver_Age  \
0               NaN                 Wet          Car        51.0   
1          Moderate                 Wet        Truck        49.0   
2               Low                 Icy          Car        54.0   
3               Low  Under Construction          Bus        34.0   
4               Low                 Dry          Car        62.0   

   Driver_Experience Road_Light_Condition  ...  Weather_Snowy  Weather_Stormy  \
0               48.0     Artificial Light  ...          False           False   
1 

Speed Limit

In [88]:
df_copy['Speed_Limit'].isna().sum()

42

In [89]:
df_copy_2 = df_copy.copy()
df_copy_2.Speed_Limit = df_copy_2.Speed_Limit.fillna(60)
df_copy_2.Speed_Limit

,Speed_Limit
0,100.0
1,120.0
2,60.0
3,60.0
4,195.0
...,...
835,30.0
836,60.0
837,30.0
838,60.0


In [90]:
df_copy_2['Speed_Limit'].isna().sum()

0

In [91]:
df_copy_2.head()

,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,Road_Light_Condition,...,Weather_Snowy,Weather_Stormy,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,Artificial Light,...,False,False,True,False,False,False,False,False,True,False
1,NaN,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,Artificial Light,...,False,False,False,False,False,True,False,False,False,True
2,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,Artificial Light,...,False,False,False,True,False,False,False,True,False,False
3,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,Daylight,...,False,False,True,False,False,False,True,False,False,False
4,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,Artificial Light,...,False,False,False,True,False,False,False,False,True,False


Traffic Density

In [92]:
df_copy_2.Traffic_Density.value_counts()

,count
Traffic_Density,
1.0,307
2.0,246
0.0,245


In [93]:
df_copy_2.Traffic_Density = df_copy_2.Traffic_Density.fillna(1)

In [94]:
print(df_copy.Traffic_Density.isna().sum())
print(df_copy_2.Traffic_Density.isna().sum())

42
0


In [95]:
df_copy_2

,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,Road_Light_Condition,...,Weather_Snowy,Weather_Stormy,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,Artificial Light,...,False,False,True,False,False,False,False,False,True,False
1,1.0,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,Artificial Light,...,False,False,False,False,False,True,False,False,False,True
2,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,Artificial Light,...,False,False,False,True,False,False,False,True,False,False
3,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,Daylight,...,False,False,True,False,False,False,True,False,False,False
4,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,Artificial Light,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,2.0,30.0,4.0,0.0,Low,Dry,Car,23.0,15.0,Daylight,...,False,False,False,True,False,False,False,False,False,True
836,2.0,60.0,4.0,0.0,Low,Dry,Motorcycle,52.0,46.0,Daylight,...,False,False,False,False,False,True,False,True,False,False
837,1.0,30.0,4.0,1.0,High,Dry,Car,NaN,34.0,Artificial Light,...,False,False,False,True,False,False,False,True,False,False
838,2.0,60.0,3.0,0.0,Low,Dry,Car,25.0,19.0,Artificial Light,...,False,False,False,True,False,False,True,False,False,False


No of vehicles

In [96]:
df_copy_2.Number_of_Vehicles.value_counts()

,count
Number_of_Vehicles,
3.0,163
4.0,161
5.0,154
2.0,153
1.0,144
11.0,8
10.0,5
13.0,4
14.0,4


In [97]:
print(df_copy_2.Number_of_Vehicles.isna().sum())
df_copy_2.Number_of_Vehicles = df_copy_2.Number_of_Vehicles.fillna(3)
print(df_copy_2.Number_of_Vehicles.isna().sum())

42
0


In [98]:
df_copy_2

,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,Road_Light_Condition,...,Weather_Snowy,Weather_Stormy,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,1.0,100.0,5.0,0.0,NaN,Wet,Car,51.0,48.0,Artificial Light,...,False,False,True,False,False,False,False,False,True,False
1,1.0,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,Artificial Light,...,False,False,False,False,False,True,False,False,False,True
2,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,Artificial Light,...,False,False,False,True,False,False,False,True,False,False
3,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,Daylight,...,False,False,True,False,False,False,True,False,False,False
4,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,Artificial Light,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,2.0,30.0,4.0,0.0,Low,Dry,Car,23.0,15.0,Daylight,...,False,False,False,True,False,False,False,False,False,True
836,2.0,60.0,4.0,0.0,Low,Dry,Motorcycle,52.0,46.0,Daylight,...,False,False,False,False,False,True,False,True,False,False
837,1.0,30.0,4.0,1.0,High,Dry,Car,NaN,34.0,Artificial Light,...,False,False,False,True,False,False,False,True,False,False
838,2.0,60.0,3.0,0.0,Low,Dry,Car,25.0,19.0,Artificial Light,...,False,False,False,True,False,False,True,False,False,False


Driver_Alcohol

In [99]:
df_copy_2.Driver_Alcohol.value_counts()

,count
Driver_Alcohol,
0.0,670
1.0,128


In [100]:
print(df_copy_2.Driver_Alcohol.isna().sum())
df_copy_2.Driver_Alcohol = df_copy_2.Driver_Alcohol.fillna(0)
print(df_copy_2.Driver_Alcohol.isna().sum())

42
0


Accident_Severity

In [101]:
df_copy_2.Accident_Severity.value_counts()

,count
Accident_Severity,
Low,478
Moderate,241
High,79


In [102]:
print(df_copy_2.Accident_Severity.isna().sum())
df_copy_2.Accident_Severity = df_copy_2.Accident_Severity.fillna("Low")
print(df_copy_2.Accident_Severity.isna().sum())

42
0


In [103]:
df_copy_2.Accident_Severity

,Accident_Severity
0,Low
1,Moderate
2,Low
3,Low
4,Low
...,...
835,Low
836,Low
837,High
838,Low


In [105]:
df_copy_2.head()

,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,Road_Light_Condition,...,Weather_Snowy,Weather_Stormy,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,1.0,100.0,5.0,0.0,Low,Wet,Car,51.0,48.0,Artificial Light,...,False,False,True,False,False,False,False,False,True,False
1,1.0,120.0,3.0,0.0,Moderate,Wet,Truck,49.0,43.0,Artificial Light,...,False,False,False,False,False,True,False,False,False,True
2,1.0,60.0,4.0,0.0,Low,Icy,Car,54.0,52.0,Artificial Light,...,False,False,False,True,False,False,False,True,False,False
3,2.0,60.0,3.0,0.0,Low,Under Construction,Bus,34.0,31.0,Daylight,...,False,False,True,False,False,False,True,False,False,False
4,1.0,195.0,11.0,0.0,Low,Dry,Car,62.0,55.0,Artificial Light,...,False,False,False,True,False,False,False,False,True,False


In [108]:
# prompt: in df_copy_2, for Accident_Severity, replace Low with 0, Medium with 1, High with 2

# Replace Accident_Severity values
df_copy_2['Accident_Severity'] = df_copy_2['Accident_Severity'].replace({'Low': 0, 'Moderate': 1, 'High': 2})


<ipython-input-108-e264ab8cb635>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy_2['Accident_Severity'] = df_copy_2['Accident_Severity'].replace({'Low': 0, 'Moderate': 1, 'High': 2})


In [109]:
df_copy_2['Accident_Severity']

,Accident_Severity
0,0
1,1
2,0
3,0
4,0
...,...
835,0
836,0
837,2
838,0


In [110]:
df_copy_2

,Traffic_Density,Speed_Limit,Number_of_Vehicles,Driver_Alcohol,Accident_Severity,Road_Condition,Vehicle_Type,Driver_Age,Driver_Experience,Road_Light_Condition,...,Weather_Snowy,Weather_Stormy,Road_City_Road,Road_Highway,Road_Mountain_Road,Road_Rural_Road,Time_Afternoon,Time_Evening,Time_Morning,Time_Night
0,1.0,100.0,5.0,0.0,0,Wet,Car,51.0,48.0,Artificial Light,...,False,False,True,False,False,False,False,False,True,False
1,1.0,120.0,3.0,0.0,1,Wet,Truck,49.0,43.0,Artificial Light,...,False,False,False,False,False,True,False,False,False,True
2,1.0,60.0,4.0,0.0,0,Icy,Car,54.0,52.0,Artificial Light,...,False,False,False,True,False,False,False,True,False,False
3,2.0,60.0,3.0,0.0,0,Under Construction,Bus,34.0,31.0,Daylight,...,False,False,True,False,False,False,True,False,False,False
4,1.0,195.0,11.0,0.0,0,Dry,Car,62.0,55.0,Artificial Light,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,2.0,30.0,4.0,0.0,0,Dry,Car,23.0,15.0,Daylight,...,False,False,False,True,False,False,False,False,False,True
836,2.0,60.0,4.0,0.0,0,Dry,Motorcycle,52.0,46.0,Daylight,...,False,False,False,False,False,True,False,True,False,False
837,1.0,30.0,4.0,1.0,2,Dry,Car,NaN,34.0,Artificial Light,...,False,False,False,True,False,False,False,True,False,False
838,2.0,60.0,3.0,0.0,0,Dry,Car,25.0,19.0,Artificial Light,...,False,False,False,True,False,False,True,False,False,False


# Decision Trees From Scratch